In [ ]:
import numpy as np
import scipy.stats as sts
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('ggplot')

from tqdm.auto import tqdm

In [ ]:
# функция бутстрапа

def get_bootstrap(
    data_column_1, # числовые значения первой выборки
    data_column_2, # числовые значения второй выборки
    boot_it = 1000, # количество бутстрэп-подвыборок
    statistic = np.mean, # интересующая нас статистика
    bootstrap_conf_level = 0.95 # уровень значимости
):
    boot_data = []
    for i in tqdm(range(boot_it)): # извлекаем подвыборки
        samples_1 = data_column_1.sample(
            len(data_column_1), # чтобы сохранить дисперсию, берем такой же размер выборки
            replace = True # параметр возвращения
        ).values
        
        samples_2 = data_column_2.sample(
            len(data_column_2), 
            replace = True
        ).values
        
        boot_data.append(statistic(samples_1-samples_2)) # mean() - применяем статистику
        
    pd_boot_data = pd.DataFrame(boot_data)
        
    left_quant = (1 - bootstrap_conf_level)/2
    right_quant = 1 - (1 - bootstrap_conf_level) / 2
    quants = pd_boot_data.quantile([left_quant, right_quant])
        
    p_1 = norm.cdf(
        x = 0, 
        loc = np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_2 = norm.cdf(
        x = 0, 
        loc = -np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_value = min(p_1, p_2) * 2
        
    # Визуализация
    _, _, bars = plt.hist(pd_boot_data[0], bins = 50)
    for bar in bars:
        if abs(bar.get_x()) <= quants.iloc[0][0] or abs(bar.get_x()) >= quants.iloc[1][0]:
            bar.set_facecolor('red')
        else: 
            bar.set_facecolor('grey')
            bar.set_edgecolor('black')
    
    plt.style.use('ggplot')
    plt.vlines(quants,ymin=0,ymax=50,linestyle='--')
    plt.xlabel('boot_data')
    plt.ylabel('frequency')
    plt.title("Histogram of boot_data")
    plt.show()
       
    return {"boot_data": boot_data, 
            "quants": quants, 
            "p_value": p_value}

In [ ]:
np.random.seed(2)

## Бакетирование ----
Трансформация распределения случайной величины в нормальное с помощью техники бакетирования

Бакетирование подходит тогда, когда необходимо:
* сохранить информацию о дисперсии и среднем в выборке до трансформации
* привести к нормальному распределению

Сгенерируем данные. В качестве примера используется эксп. распределение

Возьмем кратное количество групп. Допустим 5000 (можно взять и 200, и 500)

In [ ]:
b_n = 5000
n = 100000

val_1 = np.random.exponential(scale=1/0.01, size=n)
val_2 = np.random.exponential(scale=1/0.011, size=n)

sample_exp = pd.DataFrame({
    "values":   np.concatenate([val_1, val_2]),  
    "variant":  ["A" for i in range(n)] + ["B" for i in range(n)],
    "backet":   [i for i in range(b_n)] * int(n*2/b_n)
})

In [ ]:
sample_exp.head()

In [ ]:
# Группируем по бакетам и считаем среднее:

backeted_sample_exp = sample_exp.groupby(by=["backet","variant"])["values"].agg(
    mu=np.mean, 
    sd_mu=np.std
).reset_index()

In [ ]:
backeted_sample_exp.head()

In [ ]:
# Сравним исходное выборочное среднее и среднее бакетных средних 
# Будет TRUE
round(np.mean(sample_exp["values"]),5) == round(np.mean(backeted_sample_exp["mu"]),5)

In [ ]:
# Проверим дисперсию. Отнормируем по кол-ву наблюдений
np.var(sample_exp["values"]) / len(sample_exp["values"])

In [ ]:
# Дисперсия будет почти такой же
np.var(backeted_sample_exp["mu"]) / len(backeted_sample_exp["mu"])

In [ ]:
# исходное распределение
viz = sample_exp["values"].plot(kind="hist", color="grey", figsize=(8,5), bins=50)
viz.set_xlabel("values")
viz.set_ylabel("count")

In [ ]:
# Распределение после бакетного преобразования
viz = backeted_sample_exp["mu"].plot(kind="hist", color="grey", figsize=(8,5), bins=50)
viz.set_xlabel("mu")
viz.set_ylabel("count")

In [ ]:
%%time
# Сравним результаты для бутстрапа оригинального семпла и бакетного
# оригинал
original_booted_data = get_bootstrap(
    sample_exp[sample_exp.variant=="A"]["values"],
    sample_exp[sample_exp.variant=="B"]["values"]
)

In [ ]:
%%time

# бакеты
backeted_booted_data = get_bootstrap( 
    backeted_sample_exp[backeted_sample_exp.variant=="A"]["mu"],
    backeted_sample_exp[backeted_sample_exp.variant=="B"]["mu"],
  ) 

In [ ]:
# Сравним результаты
# Дисперсия
print(np.var(original_booted_data["boot_data"]))
print(np.var(backeted_booted_data["boot_data"]))

In [ ]:
# ДИ
original_booted_data["quants"]

In [ ]:
backeted_booted_data["quants"]